# Lane Follow

In this example, we will use our trained model to make the robot move smoothly on the track.

## Loading the model

Using ``best_steering_model_xy.pth``



In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

In [ ]:
from servoserial import ServoSerial
import threading
# Kill thpread
import inspect
import ctypes
import ipywidgets.widgets as widgets
from IPython.display import display
import time

controller = widgets.Controller(index=0)
display(controller)

Controller()

Load the trained model ``best_steering_model_xy.pth``.

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

CPU to GPU

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

# Pre-processing function
Match the format of the camera
1. Convert from HWC(Channel, Height, Width) layout to CHW layout
2. Normalize using the same parameters (The camera provides values in the range [0, 255], and the training loaded image is in the range [0, 1], so we need to scale 255.0
3. Transfer data from CPU memory to GPU memory
4. Add a batch dimension

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

We have defined a preprocessing function that converts images from camera format to neural network input format.

start showing our camera.

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg
from servoserial import ServoSerial

#camera = Camera()
camera = Camera.instance(width=224, height=224, fps=10)
servo_device = ServoSerial()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

def camservoInitFunction():
    global leftrightpulse, updownpulse
    leftrightpulse = 2048
    updownpulse = 1500
    servo_device.Servo_serial_control(1, 2048)
    time.sleep(0.1)
    servo_device.Servo_serial_control(2, 1300)

serial Open!


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Create a robot instance to drive the motor

In [ ]:
from jetbot import Robot

robot = Robot()

Sliders to control Speed and Steering

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.75,description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.33, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.12, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.75, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.33, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.12, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

The x and y sliders will display the predicted x, y values. The steering slider will display our estimated steering value. This value is not the actual angle of the target, but an almost proportional value.

When the actual angle is ``0``, this is 0, which will increase/decrease as the actual angle increases/decreases:

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

Function to:-

1) Preprocess camera image
2) Perform a neural network
3) Calculate the approximate steering value
4) Control the motor using proportional/differential control (PD)

In [ ]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0

    x_slider.value = x
    y_slider.value = y

    speed_slider.value = speed_gain_slider.value

    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle

    steering_slider.value = pid + steering_bias_slider.value

    #PID+ Base speed + Gain
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

execute({'new': camera.value})

Linking the neural netwrok to the camera for processing.

In [ ]:
camservoInitFunction()
camera.observe(execute, names='value')

184
b'\xff\xff\x01\x07\x03*\x08\x00\x00\n\xb8'
166
b'\xff\xff\x02\x07\x03*\x05\x14\x00\n\xa6'


In [ ]:
camera.unobserve(execute, names='value')
time.sleep(0.1)
robot.stop()